In [15]:
import sys
import os

sys.path.append(os.path.abspath('../tapeagents'))

from tapeagents.llms import OpenrouterLLM, LLMOutput

api_key = "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"
llm = OpenrouterLLM(
    model_name="meta-llama/llama-3.3-70b-instruct:free",  # https://openrouter.ai/meta-llama/llama-3.3-70b-instruct:free
    api_token=api_key,
    parameters={"temperature": 0.1},
)

In [ ]:
regulations_info = open()

In [ ]:
from tapeagents.agent import Agent
from tapeagents.core import PartialStep, Prompt, TapeMetadata
from tapeagents.dialog_tape import AssistantStep, DialogTape, SystemStep, UserStep

import ipywidgets as widgets
from IPython.display import display, clear_output

class ChildcareSupportAgent(Agent[DialogTape]):

    def make_prompt(self, tape: DialogTape):
        return Prompt(messages=[{"role": "system", "content": tape.steps[0].content}] + [
            {"role": "user" if isinstance(step, UserStep) else "assistant", "content": step.content}
            for step in tape.steps[1:]
        ])
        
    def generate_steps(self, tape: DialogTape, llm_stream):
        buffer = []
        for event in llm_stream:  # This is a generator of chunks and final outputs
            if event.chunk:
                buffer.append(event.chunk)
                yield PartialStep(step=AssistantStep(content="".join(buffer)))
            elif event.output:
                yield AssistantStep(content=event.output.content or "")
                return
            else:
                raise ValueError(f"Unknown event type from LLM: {event}")

    def make_llm_output(self, tape: DialogTape, index: int):
        step = tape.steps[index]
        if not isinstance(step, AssistantStep):
            raise ValueError("Expected AssistantStep")
        return LLMOutput(content=step.content)


# Create agent instance
agent = ChildcareSupportAgent.create(llm, name="childcare_agent")

flag_guide = """
Childcare Application Flags:
- Flag A: Missing proof of income
- Flag B: Child not registered in municipal system
- Flag C: Application submitted after deadline
- Flag D: Inconsistent residency records

Use this guide to explain flags to municipality workers.
"""

# Initialize tape with system context (childcare domain prompt)
tape = DialogTape(
    context=None,
    steps=[SystemStep(content=(
        "You are a childcare support assistant for municipality workers reviewing applications. "
        "You explain application flags and decisions made by an external deterministic program." + flag_guide
    ))]
)


input_box = widgets.Text(placeholder="Ask me anything")
output_area = widgets.Output()

display(input_box, output_area)

def handle_user_input(submit_event):
    global tape
    user_input = submit_event.value
    input_box.value = ""  # clear input

    with output_area:
        print(f"👤 You: {user_input}")
        tape = tape.append(UserStep(content=user_input))
        # print(f"🤖 Assistant: ", end="", flush=True)

        new_tape = None
        for event in agent.run(tape):
            if event.partial_step:
                print(event.partial_step.step.content, end="", flush=True)
            if event.final_tape:
                new_tape = event.final_tape
        tape = new_tape
        last_step = tape.steps[-1]
        print(f"🤖 Assistant: {last_step.content}")
        print("\n")

# for step in tape.steps:
#     print(step.__class__.__name__, ":", step.content)

input_box.on_submit(handle_user_input)

Text(value='', placeholder='Ask me anything')

Output()

/var/folders/z8/ydl_c8q95yv6xh0g25nkj3n40000gn/T/ipykernel_48699/3671047229.py:99: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  input_box.on_submit(handle_user_input)
